In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from numpy.typing import NDArray
from scipy.optimize import minimize
from typing import List, Tuple
import time as time
import pandas as pd
from sklearn.metrics import r2_score

In [ ]:
# 価格を生成する関数
def create_price(r_mean: float, r_std: float, M: int) -> NDArray[np.float_]:
    # r_mean = (r_min + r_max) / 2
    # r_std = (r_max - r_mean) / 2
    # r_minとr_maxの間のランダムな0.1刻みの少数をM個生成

    # 平均r_meanと標準偏差r_stdを指定して正規分布に従うM個の価格を生成
    price = np.random.normal(r_mean, r_std, size=M)
    # price = np.round(price, 1)

    return price


# alphaを作成する関数
def alpha_star(M: int) -> NDArray[np.float_]:

    # alphaはM個の要素を持つベクトルで、各要素は[-3M, 3M]の範囲で一様分布から生成
    alpha_star = np.random.uniform(M, 3 * M, size=M)
    return alpha_star


# betaを作成する関数
def beta_star(M: int, M_prime: int) -> NDArray[np.float_]:

    # betaはM x M_primeのゼロ行列を作成
    beta_star = np.zeros((M, M_prime))

    for m in range(M):
        for m_prime in range(M_prime):
            # mとm_primeが同じ場合は[-3M, -2M]の範囲で一様分布から生成
            if m == m_prime:
                beta_star[m, m_prime] = np.random.uniform(-3 * M, -2 * M)
            # mとm_primeが異なる場合は[0, 3]の範囲で一様分布から生成
            else:
                beta_star[m, m_prime] = np.random.uniform(0, 3)

    return beta_star


def quantity_function(
    price: NDArray[np.float_],
    alpha: NDArray[np.float_],
    beta: NDArray[np.float_],
    delta: float = 0.1,  # ノイズレベルを指定（例として0.1を使用）
) -> list[float]:
    M = len(price)
    quantity_list = []
    q_m_no_noise = []

    # ステップ1: ノイズなしのq_mを計算
    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]
        quantity = alpha[m] + sum_beta
        q_m_no_noise.append(quantity)

    # E[q_m^2]を計算
    E_q_m_squared = np.mean(np.array(q_m_no_noise) ** 2)

    # ステップ2: ノイズの標準偏差sigmaを計算
    sigma = delta * np.sqrt(E_q_m_squared)

    # ステップ3: ノイズを加えて最終的なq_mを計算
    for m in range(M):
        epsilon = np.random.normal(0, sigma)
        quantity = q_m_no_noise[m] + epsilon
        quantity_list.append(quantity)

    return quantity_list


def sales_function(
    price: NDArray[np.float_], alpha: NDArray[np.float_], beta: NDArray[np.float_]
) -> list[float]:
    M = len(price)
    sales_list = []
    # ノイズなしのq_mを計算
    for m in range(M):
        sum_beta = 0
        for m_prime in range(M):
            sum_beta += beta[m][m_prime] * price[m_prime]

        quantity = alpha[m] + sum_beta

        # 需要量に価格をかけて売上を計算
        sales_list.append(quantity * price[m])

    return sales_list


def create_date(M, N, r_mean, r_std, delta=0.1):

    # alphaとbetaを生成
    alpha = alpha_star(M)
    beta = beta_star(M, M)

    # 価格のリストを作成
    price_list = []
    # 需要のリストを作成
    quantity_list = []

    for _ in range(N):

        # 価格を作成
        price = create_price(r_mean, r_std, M)
        # 需要を計算
        quantity = quantity_function(price, alpha, beta, delta)
        # リストに追加
        price_list.append(price)
        quantity_list.append(quantity)
    # 価格と需要をDataFrameに変換
    X = np.array(price_list)
    Y = np.array(quantity_list)

    return alpha, beta, X, Y


def create_bounds(M, r_min, r_max):
    # 価格の下限を設定
    lb = np.full(M, r_min)
    # 価格の上限を設定
    ub = np.full(M, r_max)

    # 提案手法にれる価格範囲
    range_bounds = []
    for i in range(M):
        range_bounds.append(lb[i])

    for i in range(M):
        range_bounds.append(ub[i])
    # 一般的な価格範囲
    bounds = [(r_min, r_max) for _ in range(M)]

    return lb, ub, bounds, range_bounds

In [8]:
# 目的関数を定義（最大化問題を最小化問題に変換）
def sales_objective_function(prices, alpha, beta, M):
    return -sum(
        prices[m] * (alpha[m] + sum(beta[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


def sales_optimize(
    M: int,
    alpha: np.ndarray,
    beta: np.ndarray,
    bounds: list[tuple[float, float]],
) -> Tuple[float, np.ndarray]:
    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)

    # 最適化を実行
    result = minimize(
        sales_objective_function,
        initial_prices,
        args=(alpha, beta, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices

In [3]:
# 目的関数を定義
def predict_objective_function(
    prices: NDArray[np.float_], intercepts: [float], coefs: [NDArray[np.float_]], M: int
) -> float:
    # 各変数の内容をデバッグ出力
    # print("prices:", prices)
    # print("intercepts:", intercepts)
    # print("coefs:", coefs)
    # print("M:", M)

    return -sum(
        prices[m]
        * (intercepts[m] + sum(coefs[m][m_prime] * prices[m_prime] for m_prime in range(M)))
        for m in range(M)
    )


# 予測と最適化を行う関数
def predict_optimize(
    M: int, X: NDArray[np.float_], Y: NDArray[np.float_], bounds: list[float]
) -> tuple[float, NDArray[np.float_]]:
    lr = MultiOutputRegressor(LinearRegression())
    lr.fit(X, Y)
    # 係数と切片を取得
    coefs = [estimate.coef_ for estimate in lr.estimators_]
    intercepts = [estimate.intercept_ for estimate in lr.estimators_]

    # 初期値として与えられたprices_listを使用
    initial_prices = np.full(M, 0.6)
    # 最適化を実行
    result = minimize(
        predict_objective_function,
        initial_prices,
        args=(intercepts, coefs, M),
        bounds=bounds,
        method="L-BFGS-B",
    )
    # 最適な価格と目的関数の値を取得
    optimal_prices = result.x
    optimal_value = -result.fun  # 符号を反転して元の最大化問題の値に戻す
    return optimal_value, optimal_prices

In [4]:
# CVを行う関数
def cross_validation(
    tilda_coefs_list: list[NDArray[np.float_]],
    tilda_intercepts_list: list[float],
    hat_coefs_list: list[NDArray[np.float_]],
    hat_intercepts_list: list[float],
    M: int,
    K: int,
    bounds: list[float],
) -> float:
    optimal_sales_list = []
    optimal_prices_list = [[] for _ in range(M)]
    for i in range(K):
        # 初期値として与えられたprices_listを使用
        initial_prices = np.full(M, 0.6)

        # 最適化を実行
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(tilda_intercepts_list[i], tilda_coefs_list[i], M),
            bounds=bounds,
            method="L-BFGS-B",
        )
        # 最適な価格と目的関数の値を取得
        optimal_prices = result.x
        # print("optimal_prices cv:", optimal_prices)
        for m in range(M):
            optimal_prices_list[m].append(optimal_prices[m])

        sales_hat = np.sum(
            sales_function(optimal_prices, hat_intercepts_list[i], hat_coefs_list[i])
        )

        optimal_sales_list.append(sales_hat)

    return np.mean(optimal_sales_list), optimal_prices_list

In [ ]:
# 目的関数を定義
def cross_validation_bounds_penalty_all(
    bounds: List[float],
    tilda_coefs_list: List[NDArray[np.float_]],
    tilda_intercepts_list: List[NDArray[np.float_]],
    hat_coefs_list: List[NDArray[np.float_]],
    hat_intercepts_list: List[NDArray[np.float_]],
    M: int,
    K: int,
    bounds_range: float,
    lamda_1: float,
    lamda_2:float,
) -> float:
    # bounds の整合性チェック
    bounds_list = []
    for i in range(M):
        # 上下限が逆転していたら平均で固定
        if bounds[i] > bounds[i + M]:
            bounds_mean = (bounds[i] + bounds[i + M]) / 2
            bounds_list.append((bounds_mean, bounds_mean))
        # 上下限が逆転していなければそのまま使用
        else:
            bounds_list.append((bounds[i], bounds[i + M]))
    optimal_sales_list = []

    # すでに外部でKFold分割や学習が終わっているものとして
    # tilda_coefs_list[i], tilda_intercepts_list[i], hat_coefs_list[i], hat_intercepts_list[i]
    # を使用して最適化と売上計算
    for i in range(K):
        intercepts = tilda_intercepts_list[i]
        coefs = tilda_coefs_list[i]

        # 最適化
        initial_prices = np.full(M, 0.6)
        result = minimize(
            predict_objective_function,
            initial_prices,
            args=(intercepts, coefs, M),
            bounds=bounds_list,
            method="L-BFGS-B",
        )
        optimal_prices = result.x

        # hatモデルパラメータで売上計算
        alpha = hat_intercepts_list[i]
        beta = hat_coefs_list[i]

        sales_hat = np.sum(sales_function(optimal_prices, alpha, beta))
        optimal_sales_list.append(sales_hat)

    # ペナルティ計算
    penalty_1 = 0.0
    for i in range(M):
        penalty_1 += bounds[i + M] - bounds[i]

    penalty_2 = 0.0
    for i in range(M):
        penalty_2 += max(0,bounds[i]-bounds[i+M])**2

    mean_sales = np.mean(optimal_sales_list)

    return -mean_sales + lamda_1 * max(0, penalty_1 - M * bounds_range) ** 2 + lamda_2 * penalty_2

# 目的関数を最適化し価格範囲を推定
def estimate_bounds_penalty_nelder_all(
    bounds: List[float],
    tilda_coefs_list: List[NDArray[np.float_]],
    tilda_intercepts_list: List[NDArray[np.float_]],
    hat_coefs_list: List[NDArray[np.float_]],
    hat_intercepts_list: List[NDArray[np.float_]],
    M: int,
    K: int,
    r_min: float,
    r_max: float,
    bounds_range: float,
    lamda_1: float,
    lamda_2: float,
    adaptive: bool = True,
) -> Tuple[float, List[Tuple[float, float]]]:
    # Nelder-Meadでの最適化
    bounds_nelder = minimize(
        cross_validation_bounds_penalty_all,
        bounds,
        args=(
            tilda_coefs_list,
            tilda_intercepts_list,
            hat_coefs_list,
            hat_intercepts_list,
            M,
            K,
            bounds_range,
            lamda_1,
            lamda_2,
        ),
        method="Nelder-Mead",
        bounds=[(r_min, r_max) for _ in range(2 * M)],
        options={"adaptive": adaptive},
    )

    opt_bounds = []
    for i in range(M):
        lb = min(bounds_nelder.x[i], bounds_nelder.x[i + M])
        ub = max(bounds_nelder.x[i], bounds_nelder.x[i + M])
        opt_bounds.append((lb, ub))

    return -bounds_nelder.fun, opt_bounds

In [ ]:
def bound_quan(price_list: np.ndarray, q: float) -> Tuple[np.ndarray, np.ndarray]:
    """
    price_list : shape (N, M) の価格データ (N 件のサンプル、M 商品分)
    q          : 上限とする分位数 (例: 0.95 など)

    戻り値:
        lower_bound : (1-q) 分位数 (shape (M,))
        upper_bound : q 分位数   (shape (M,))
    """
    # axis=0 で列ごとに分位数を計算

    # 下限となる分位数を求める
    lower_bound = np.quantile(price_list, 1-q, axis=0)
    # 上限となる分位数を求める
    upper_bound = np.quantile(price_list, q, axis=0)
    bounds_quan = []
    for i in range(len(lower_bound)):
        bounds_quan.append((lower_bound[i], upper_bound[i]))
    return bounds_quan

In [7]:
def bootstrap_bounds(
    M: int,
    X: np.ndarray,
    Y: np.ndarray,
    r_min: float,
    r_max: float,
    n_iterations: int = 1000,
    k: float = 1.96
) -> tuple[np.ndarray, np.ndarray]:
    """
    ブートストラップサンプルを用いて各商品の最適価格の統計量（平均±k*標準偏差）から価格範囲を算出する関数
    
    Parameters:
      M: 商品数（価格の次元数）
      X: 価格設定のデータ（各行が一つの実験データ、shape=(n_samples, M)）
      Y: 需要のデータ（Xと対応するデータ、shape=(n_samples, M)）
      bounds: 最適化に使用する各商品の価格下限・上限のリスト（例：[(r_min, r_max), ...]）
      n_iterations: ブートストラップの反復回数（デフォルトは1000）
      k: 標準偏差のスケールパラメータ（例：1.96 なら約95%信頼区間）
    
    Returns:
      lower_bounds: 各商品の価格下限（mean - k * std）
      upper_bounds: 各商品の価格上限（mean + k * std）
      
    ※ 内部で predict_optimize 関数を使用して最適価格を算出している前提です。
    """
    bounds = [(r_min, r_max) for _ in range(M)]
    optimal_prices_list = []
    n_samples = X.shape[0]
    
    for i in range(n_iterations):
        # ブートストラップサンプルを行単位の復元抽出で取得
        indices = np.random.choice(n_samples, size=n_samples, replace=True)
        X_bs = X[indices]
        Y_bs = Y[indices]
        
        # 取得したブートストラップサンプルを用いて価格最適化を実施
        # predict_optimize は (optimal_value, optimal_prices) を返す前提
        _, opt_prices = predict_optimize(M, X_bs, Y_bs, bounds)
        optimal_prices_list.append(opt_prices)
    
    # ブートストラップで得られた最適価格を NumPy 配列に変換（shape: (n_iterations, M)）
    #print(optimal_prices_list)
    optimal_prices_array = np.array(optimal_prices_list)
    
    # 各商品の最適価格の平均と標準偏差を計算
    mean_prices = np.mean(optimal_prices_array, axis=0)
    std_prices = np.std(optimal_prices_array, axis=0)
    
    # 平均 ± k * 標準偏差を下限・上限として算出
    lower_bounds = mean_prices - k * std_prices
    upper_bounds = mean_prices + k * std_prices
    
    # 結果をタプルに格納
    bootstrap_bounds = []
    for i in range(M):
        
        # r_min と r_max でクリッピング
        lower = max(r_min, lower_bounds[i])
        upper = min(r_max, upper_bounds[i])

        bootstrap_bounds.append((lower, upper))

    return bootstrap_bounds

In [ ]:
# ----- 以下はお使いの既存の関数群を想定 -----
# create_date, create_bounds, sales_optimize, predict_optimize,
# sales_function, bound_quan, estimate_bounds_penalty_nelder_all,
# cross_validation などは、既に定義済みとして進めます。

# 実験設定
M = 5
K = 5
N = 500
B=100
r_mean = 0.8
r_std = 0.1
r_min = 0.5
r_max = 1.1
delta = 0.6
z_range = 0.4

lb, ub, bounds, range_bounds = create_bounds(M, r_min, r_max)

# 実験を何回行うか（例：2回）
num_experiments = 100

so_sales_list_5 = []
po_sales_list_5 = []
true_po_sales_list_5 = []

ebz_po_sales_list_5 = []
true_ebz_po_sales_list_5 = []
ebz_range_list_5 = []

quan95_sales_list_5 = []
true_quan95_sales_list_5 = []
quan95_range_list_5 = []

quan90_sales_list_5 = []
true_quan90_sales_list_5 = []
quan90_range_list_5 = []

quan85_sales_list_5 = []
true_quan85_sales_list_5 = []
quan85_range_list_5 = []

quan80_sales_list_5 = []
true_quan80_sales_list_5 = []
quan80_range_list_5 = []

boot99_sales_list_5 = []
true_boot99_sales_list_5 = []
boot99_range_list_5 = []

boot95_sales_list_5 = []
true_boot95_sales_list_5 = []
boot95_range_list_5 = []

boot90_sales_list_5 = []
true_boot90_sales_list_5 = []
boot90_range_list_5 = []


ebpa3_po_sales_list_5 = []
true_ebpa3_po_sales_list_5 = []
ebpa3_range_list_5 = []

ebpa4_po_sales_list_5 = []
true_ebpa4_po_sales_list_5 = []
ebpa4_range_list_5 = []

ebpa5_po_sales_list_5 = []
true_ebpa5_po_sales_list_5 = [] 
ebpa5_range_list_5 = []

ebz_range_deff_list_5 = []
ebpa3_range_deff_list_5 = []
ebpa4_range_deff_list_5 = []
ebpa5_range_deff_list_5 = []

# ★ 追加： 全体学習モデルの R^2 を記録するリスト
all_r2_list = []

each_range_deff_3_list = []
each_range_deff_4_list = []
each_range_deff_5_list = []
each_range_deff_6_list = []

for i in range(num_experiments):
    # 乱数シードを設定
    np.random.seed(i + 6)
    # データを作成
    alpha, beta, X, Y = create_date(M, N, r_mean, r_std, delta)

    # 交差検証用に切片と係数を格納するlistを作成
    tilda_coefs_list = []
    tilda_intercepts_list = []
    hat_coefs_list = []
    hat_intercepts_list = []

    # ----------- 1) KFold で tilda / hat を推定 (境界推定用) -----------
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # lr_tilda: trainデータで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())
        lr_tilda.fit(X_train, y_train)

        # tildaの係数・切片を保存
        coefs = [est.coef_ for est in lr_tilda.estimators_]
        intercepts = [est.intercept_ for est in lr_tilda.estimators_]
        tilda_coefs_list.append(coefs)
        tilda_intercepts_list.append(intercepts)

        # lr_hat: testデータで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        hat_coefs = [est.coef_ for est in lr_hat.estimators_]
        hat_intercepts = [est.intercept_ for est in lr_hat.estimators_]
        hat_coefs_list.append(hat_coefs)
        hat_intercepts_list.append(hat_intercepts)

    # ----------- 3) 以下、既存の最適化ロジック -----------
    # SO (真のパラメータを使った最適化)
    so_sales, so_prices = sales_optimize(M, alpha, beta, bounds)

    # PO (実データ X, Y で学習したモデルを使い最適化)
    po_sales, po_prices = predict_optimize(M, X, Y, bounds)
    true_po_sales = np.sum(sales_function(po_prices, alpha, beta))

    so_sales_list_5.append(so_sales / so_sales)
    po_sales_list_5.append(po_sales / so_sales)
    true_po_sales_list_5.append(true_po_sales / so_sales)

    # 95%分位数を上限,5%分位数を下限とした価格範囲を求める
    quan95_bounds = bound_quan(X, 0.95)
    quan95_po_sales, quan95_po_prices = predict_optimize(M, X, Y, quan95_bounds)
    true_quan95_po_sales = np.sum(sales_function(quan95_po_prices, alpha, beta))
    quan95_sales_list_5.append(quan95_po_sales / so_sales)
    true_quan95_sales_list_5.append(true_quan95_po_sales / so_sales)
    quan95_range = sum(quan95_bounds[i][1] - quan95_bounds[i][0] for i in range(M))
    quan95_range_list_5.append(quan95_range)

    # 90%分位数を上限,10%分位数を下限とした価格範囲を求める
    quan90_bounds = bound_quan(X, 0.9)
    quan90_po_sales, quan90_po_prices = predict_optimize(M, X, Y, quan90_bounds)
    true_quan90_po_sales = np.sum(sales_function(quan90_po_prices, alpha, beta))
    quan90_sales_list_5.append(quan90_po_sales / so_sales)
    true_quan90_sales_list_5.append(true_quan90_po_sales / so_sales)
    quan90_range = sum(quan90_bounds[i][1] - quan90_bounds[i][0] for i in range(M))
    quan90_range_list_5.append(quan90_range)

    # 85%分位数を上限,15%分位数を下限とした価格範囲を求める
    quan85_bounds = bound_quan(X, 0.85)
    quan85_po_sales, quan85_po_prices = predict_optimize(M, X, Y, quan85_bounds)
    true_quan85_po_sales = np.sum(sales_function(quan85_po_prices, alpha, beta))
    quan85_sales_list_5.append(quan85_po_sales / so_sales)
    true_quan85_sales_list_5.append(true_quan85_po_sales / so_sales)
    quan85_range = sum(quan85_bounds[i][1] - quan85_bounds[i][0] for i in range(M))
    quan85_range_list_5.append(quan85_range)

    # 80%分位数を上限,20%分位数を下限とした価格範囲を求める
    quan80_bounds = bound_quan(X, 0.8)
    quan80_po_sales, quan80_po_prices = predict_optimize(M, X, Y, quan80_bounds)
    true_quan80_po_sales = np.sum(sales_function(quan80_po_prices, alpha, beta))
    quan80_sales_list_5.append(quan80_po_sales / so_sales)
    true_quan80_sales_list_5.append(true_quan80_po_sales / so_sales)
    quan80_range = sum(quan80_bounds[i][1] - quan80_bounds[i][0] for i in range(M))
    quan80_range_list_5.append(quan80_range)

    # ブートストラップにより99%信頼区間を求め価格範囲とする
    boot99_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=2.576
    )
    boot99_po_sales, boot99_po_prices = predict_optimize(M, X, Y, boot99_bounds)
    true_boot99_po_sales = np.sum(sales_function(boot99_po_prices, alpha, beta))
    boot99_sales_list_5.append(boot99_po_sales / so_sales)
    true_boot99_sales_list_5.append(true_boot99_po_sales / so_sales)
    boot99_range = sum(boot99_bounds[i][1] - boot99_bounds[i][0] for i in range(M))
    boot99_range_list_5.append(boot99_range)

    # ブートストラップ法により95%信頼区間を求め価格範囲とする
    boot95_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=1.96
    )
    boot95_po_sales, boot95_po_prices = predict_optimize(M, X, Y, boot95_bounds)
    true_boot95_po_sales = np.sum(sales_function(boot95_po_prices, alpha, beta))
    boot95_sales_list_5.append(boot95_po_sales / so_sales)
    true_boot95_sales_list_5.append(true_boot95_po_sales / so_sales)
    boot95_range = sum(boot95_bounds[i][1] - boot95_bounds[i][0] for i in range(M))
    boot95_range_list_5.append(boot95_range)

    # ブートストラップ法により90%信頼区間を求め価格範囲とする
    boot90_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=1.645
    )
    boot90_po_sales, boot90_po_prices = predict_optimize(M, X, Y, boot90_bounds)
    true_boot90_po_sales = np.sum(sales_function(boot90_po_prices, alpha, beta))
    boot90_sales_list_5.append(boot90_po_sales / so_sales)
    true_boot90_sales_list_5.append(true_boot90_po_sales / so_sales)
    boot90_range = sum(boot90_bounds[i][1] - boot90_bounds[i][0] for i in range(M))
    boot90_range_list_5.append(boot90_range)


    # 提案手法による価格範囲の推定(価格範囲の上限3.0)
    ebz_val, ebz_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.6,
        1.0,
        1.0,
    )
    ebz_po_sales, ebz_po_prices = predict_optimize(M, X, Y, ebz_bounds)
    true_ebz_po_sales = np.sum(sales_function(ebz_po_prices, alpha, beta))
    ebz_po_sales_list_5.append(ebz_po_sales / so_sales)
    true_ebz_po_sales_list_5.append(true_ebz_po_sales / so_sales)
    ebz_range = sum(ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M))
    ebz_range_list_5.append(ebz_range)

    # 価格範囲の上限1.5(=0.3*5)
    ebpa3_val, ebpa3_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.30,
        1.0,
        1.0,
    )
    ebpa3_po_sales, ebpa3_po_prices = predict_optimize(M, X, Y, ebpa3_bounds)
    true_ebpa3_po_sales = np.sum(sales_function(ebpa3_po_prices, alpha, beta))
    ebpa3_po_sales_list_5.append(ebpa3_po_sales / so_sales)
    true_ebpa3_po_sales_list_5.append(true_ebpa3_po_sales / so_sales)
    ebpa3_range = sum(ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M))
    ebpa3_range_list_5.append(ebpa3_range)

    # 価格範囲の上限2.0(=0.4*5)
    ebpa4_val, ebpa4_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.40,
        1.0,
        1.0,
    )
    ebpa4_po_sales, ebpa4_po_prices = predict_optimize(M, X, Y, ebpa4_bounds)
    true_ebpa4_po_sales = np.sum(sales_function(ebpa4_po_prices, alpha, beta))
    ebpa4_po_sales_list_5.append(ebpa4_po_sales / so_sales)
    true_ebpa4_po_sales_list_5.append(true_ebpa4_po_sales / so_sales)
    ebpa4_range = sum(ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M))
    ebpa4_range_list_5.append(ebpa4_range)

    # 価格範囲の上限2.5(=0.5*5)
    ebpa5_val, ebpa5_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.50,
        1.0,
        1.0,
    )
    ebpa5_po_sales, ebpa5_po_prices = predict_optimize(M, X, Y, ebpa5_bounds)
    true_ebpa5_po_sales = np.sum(sales_function(ebpa5_po_prices, alpha, beta))
    ebpa5_po_sales_list_5.append(ebpa5_po_sales / so_sales)
    true_ebpa5_po_sales_list_5.append(true_ebpa5_po_sales / so_sales)
    ebpa5_range = sum(ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M))
    ebpa5_range_list_5.append(ebpa5_range)

    ebz_range_deff_list_ = [ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M)]
    ebpa3_range_deff_list_ = [ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M)]
    ebpa4_range_deff_list_ = [ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M)]
    ebpa5_range_deff_list_ = [ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M)]

    ebz_range_deff_list_5.append(np.sum(ebz_range_deff_list_))
    ebpa3_range_deff_list_5.append(np.sum(ebpa3_range_deff_list_))
    ebpa4_range_deff_list_5.append(np.sum(ebpa4_range_deff_list_))
    ebpa5_range_deff_list_5.append(np.sum(ebpa5_range_deff_list_))

    test_lr = MultiOutputRegressor(LinearRegression())
    test_lr.fit(X, Y)
    test_coefs = [estimate.coef_ for estimate in test_lr.estimators_]
    test_intercepts = [estimate.intercept_ for estimate in test_lr.estimators_]

    all_pred = test_lr.predict(X)
    for k, estimator in enumerate(test_lr.estimators_):
        # k番目の列（= k番目の出力）の真の値と予測値を取り出し
        y_true_k = Y[:, k]
        y_pred_k = all_pred[:, k]
        # print(f"y_true_k: {y_true_k}")
        # print(f"y_pred_k: {y_pred_k}")
        # print(y_true_k == y_pred_k)
        # print(estimator)
        # RMSE の計算
        # R^2 の計算
        r2_k = r2_score(y_true_k, y_pred_k)

        all_r2_list.append(r2_k)

    # ★ 追加：各商品の範囲の差を記録
    for j in range(M):
        each_range_deff_6_list.append(ebz_bounds[j][1] - ebz_bounds[j][0])
        each_range_deff_3_list.append(ebpa3_bounds[j][1] - ebpa3_bounds[j][0])
        each_range_deff_4_list.append(ebpa4_bounds[j][1] - ebpa4_bounds[j][0])
        each_range_deff_5_list.append(ebpa5_bounds[j][1] - ebpa5_bounds[j][0])

# -------------------------------
# 最後に全体データ学習モデルの R^2 を確認

In [ ]:
# df_5_6にso_sales_list_5からebpa_range_diff_5までのリストを追加

df_5_6 = pd.DataFrame(
    {
        "so_sales_list_5": so_sales_list_5,
        "po_sales_list_5": po_sales_list_5,
        "true_po_sales_list_5": true_po_sales_list_5,
        "ebz_po_sales_list_5": ebz_po_sales_list_5,
        "true_ebz_po_sales_list_5": true_ebz_po_sales_list_5,
        "quan95_sales_list_5": quan95_sales_list_5,
        "true_quan95_sales_list_5": true_quan95_sales_list_5,
        "quan90_sales_list_5": quan90_sales_list_5,
        "true_quan90_sales_list_5": true_quan90_sales_list_5,
        "quan85_sales_list_5": quan85_sales_list_5,
        "true_quan85_sales_list_5": true_quan85_sales_list_5,
        "quan80_sales_list_5": quan80_sales_list_5,
        "true_quan80_sales_list_5": true_quan80_sales_list_5,
        "boot99_sales_list_5": boot99_sales_list_5,
        "true_boot99_sales_list_5": true_boot99_sales_list_5,
        "boot95_sales_list_5": boot95_sales_list_5,
        "true_boot95_sales_list_5": true_boot95_sales_list_5,
        "boot90_sales_list_5": boot90_sales_list_5,
        "true_boot90_sales_list_5": true_boot90_sales_list_5,
        "ebpa3_po_sales_list_5": ebpa3_po_sales_list_5,
        "true_ebpa3_po_sales_list_5": true_ebpa3_po_sales_list_5,
        "ebpa4_po_sales_list_5": ebpa4_po_sales_list_5,
        "true_ebpa4_po_sales_list_5": true_ebpa4_po_sales_list_5,
        "ebpa5_po_sales_list_5": ebpa5_po_sales_list_5,
        "true_ebpa5_po_sales_list_5": true_ebpa5_po_sales_list_5,
        "epa6_po_sales_list_5": ebz_po_sales_list_5,
        "true_ebpa6_po_sales_list_5": true_ebz_po_sales_list_5,
        "ebpa3_range_deff_list_5": ebpa3_range_deff_list_5,
        "ebpa4_range_deff_list_5": ebpa4_range_deff_list_5,
        "ebpa5_range_deff_list_5": ebpa5_range_deff_list_5,
        "ebpa6_range_deff_list_5": ebz_range_deff_list_5,
    }
)

# csvファイルに出力
df_5_6.to_csv("df_5_6_2.csv", index=False)

In [ ]:
df_5_6_range = pd.DataFrame(
    {
    "quan95_range_list_5": quan95_range_list_5,
    "quan90_range_list_5": quan90_range_list_5,
    "quan85_range_list_5": quan85_range_list_5,
    "quan80_range_list_5": quan80_range_list_5,
    "boot99_range_list_5": boot99_range_list_5,
    "boot95_range_list_5": boot95_range_list_5,
    "boot90_range_list_5": boot90_range_list_5,
    "ebz_range_list_5": ebz_range_list_5,
    "ebpa3_range_list_5": ebpa3_range_list_5,
    "ebpa4_range_list_5": ebpa4_range_list_5,
    "ebpa5_range_list_5": ebpa5_range_list_5,
    }
)
# csvファイルに出力
df_5_6_range.to_csv("df_5_6_range_2.csv", index=False)

In [ ]:
df_5_6_each = pd.DataFrame(
    {
        "R2": all_r2_list,
        "each_range_deff_3_list": each_range_deff_3_list,
        "each_range_deff_4_list": each_range_deff_4_list,
        "each_range_deff_5_list": each_range_deff_5_list,
        "each_range_deff_6_list": each_range_deff_6_list,
    }
)

# csvファイルに出力
df_5_6_each.to_csv("df_5_6_each_2.csv", index=False)

In [ ]:
# ----- 以下はお使いの既存の関数群を想定 -----
# create_date, create_bounds, sales_optimize, predict_optimize,
# sales_function, bound_quan, estimate_bounds_penalty_nelder_all,
# cross_validation などは、既に定義済みとして進めます。

# 実験設定
M = 5
K = 5
N = 500
B=100
r_mean = 0.8
r_std = 0.1
r_min = 0.5
r_max = 1.1
delta = 0.8
z_range = 0.4

lb, ub, bounds, range_bounds = create_bounds(M, r_min, r_max)

# 実験を何回行うか（例：2回）
num_experiments = 100

so_sales_list_5 = []
po_sales_list_5 = []
true_po_sales_list_5 = []

ebz_po_sales_list_5 = []
true_ebz_po_sales_list_5 = []
ebz_range_list_5 = []

quan95_sales_list_5 = []
true_quan95_sales_list_5 = []
quan95_range_list_5 = []

quan90_sales_list_5 = []
true_quan90_sales_list_5 = []
quan90_range_list_5 = []

quan85_sales_list_5 = []
true_quan85_sales_list_5 = []
quan85_range_list_5 = []

quan80_sales_list_5 = []
true_quan80_sales_list_5 = []
quan80_range_list_5 = []

boot99_sales_list_5 = []
true_boot99_sales_list_5 = []
boot99_range_list_5 = []

boot95_sales_list_5 = []
true_boot95_sales_list_5 = []
boot95_range_list_5 = []

boot90_sales_list_5 = []
true_boot90_sales_list_5 = []
boot90_range_list_5 = []


ebpa3_po_sales_list_5 = []
true_ebpa3_po_sales_list_5 = []
ebpa3_range_list_5 = []

ebpa4_po_sales_list_5 = []
true_ebpa4_po_sales_list_5 = []
ebpa4_range_list_5 = []

ebpa5_po_sales_list_5 = []
true_ebpa5_po_sales_list_5 = [] 
ebpa5_range_list_5 = []

ebz_range_deff_list_5 = []
ebpa3_range_deff_list_5 = []
ebpa4_range_deff_list_5 = []
ebpa5_range_deff_list_5 = []

# ★ 追加： 全体学習モデルの R^2 を記録するリスト
all_r2_list = []

each_range_deff_3_list = []
each_range_deff_4_list = []
each_range_deff_5_list = []
each_range_deff_6_list = []

for i in range(num_experiments):
    np.random.seed(i + 8)
    alpha, beta, X, Y = create_date(M, N, r_mean, r_std, delta)

    tilda_coefs_list = []
    tilda_intercepts_list = []
    hat_coefs_list = []
    hat_intercepts_list = []

    # ----------- 1) KFold で tilda / hat を推定 (境界推定用) -----------
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # lr_tilda: trainデータで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())
        lr_tilda.fit(X_train, y_train)

        # tildaの係数・切片を保存
        coefs = [est.coef_ for est in lr_tilda.estimators_]
        intercepts = [est.intercept_ for est in lr_tilda.estimators_]
        tilda_coefs_list.append(coefs)
        tilda_intercepts_list.append(intercepts)

        # lr_hat: testデータで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        hat_coefs = [est.coef_ for est in lr_hat.estimators_]
        hat_intercepts = [est.intercept_ for est in lr_hat.estimators_]
        hat_coefs_list.append(hat_coefs)
        hat_intercepts_list.append(hat_intercepts)

    # ----------- 3) 以下、既存の最適化ロジック -----------
    # SO (真のパラメータを使った最適化)
    so_sales, so_prices = sales_optimize(M, alpha, beta, bounds)

    # PO (実データ X, Y で学習したモデルを使い最適化)
    po_sales, po_prices = predict_optimize(M, X, Y, bounds)
    true_po_sales = np.sum(sales_function(po_prices, alpha, beta))

    so_sales_list_5.append(so_sales / so_sales)
    po_sales_list_5.append(po_sales / so_sales)
    true_po_sales_list_5.append(true_po_sales / so_sales)

    # QUAN(95%)
    quan95_bounds = bound_quan(X, 0.95)
    quan95_po_sales, quan95_po_prices = predict_optimize(M, X, Y, quan95_bounds)
    true_quan95_po_sales = np.sum(sales_function(quan95_po_prices, alpha, beta))
    quan95_sales_list_5.append(quan95_po_sales / so_sales)
    true_quan95_sales_list_5.append(true_quan95_po_sales / so_sales)
    quan95_range = sum(quan95_bounds[i][1] - quan95_bounds[i][0] for i in range(M))
    quan95_range_list_5.append(quan95_range)

    # QUAN(90%)
    quan90_bounds = bound_quan(X, 0.9)
    quan90_po_sales, quan90_po_prices = predict_optimize(M, X, Y, quan90_bounds)
    true_quan90_po_sales = np.sum(sales_function(quan90_po_prices, alpha, beta))
    quan90_sales_list_5.append(quan90_po_sales / so_sales)
    true_quan90_sales_list_5.append(true_quan90_po_sales / so_sales)
    quan90_range = sum(quan90_bounds[i][1] - quan90_bounds[i][0] for i in range(M))
    quan90_range_list_5.append(quan90_range)

    # QUAN(85%)
    quan85_bounds = bound_quan(X, 0.85)
    quan85_po_sales, quan85_po_prices = predict_optimize(M, X, Y, quan85_bounds)
    true_quan85_po_sales = np.sum(sales_function(quan85_po_prices, alpha, beta))
    quan85_sales_list_5.append(quan85_po_sales / so_sales)
    true_quan85_sales_list_5.append(true_quan85_po_sales / so_sales)
    quan85_range = sum(quan85_bounds[i][1] - quan85_bounds[i][0] for i in range(M))
    quan85_range_list_5.append(quan85_range)

    # QUAN(80%)
    quan80_bounds = bound_quan(X, 0.8)
    quan80_po_sales, quan80_po_prices = predict_optimize(M, X, Y, quan80_bounds)
    true_quan80_po_sales = np.sum(sales_function(quan80_po_prices, alpha, beta))
    quan80_sales_list_5.append(quan80_po_sales / so_sales)
    true_quan80_sales_list_5.append(true_quan80_po_sales / so_sales)
    quan80_range = sum(quan80_bounds[i][1] - quan80_bounds[i][0] for i in range(M))
    quan80_range_list_5.append(quan80_range)

    # BOOt(99%)
    boot99_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=2.576
    )
    boot99_po_sales, boot99_po_prices = predict_optimize(M, X, Y, boot99_bounds)
    true_boot99_po_sales = np.sum(sales_function(boot99_po_prices, alpha, beta))
    boot99_sales_list_5.append(boot99_po_sales / so_sales)
    true_boot99_sales_list_5.append(true_boot99_po_sales / so_sales)
    boot99_range = sum(boot99_bounds[i][1] - boot99_bounds[i][0] for i in range(M))
    boot99_range_list_5.append(boot99_range)

    # boot(95%)
    boot95_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=1.96
    )
    boot95_po_sales, boot95_po_prices = predict_optimize(M, X, Y, boot95_bounds)
    true_boot95_po_sales = np.sum(sales_function(boot95_po_prices, alpha, beta))
    boot95_sales_list_5.append(boot95_po_sales / so_sales)
    true_boot95_sales_list_5.append(true_boot95_po_sales / so_sales)
    boot95_range = sum(boot95_bounds[i][1] - boot95_bounds[i][0] for i in range(M))
    boot95_range_list_5.append(boot95_range)

    # boot(90%)
    boot90_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=1.645
    )
    boot90_po_sales, boot90_po_prices = predict_optimize(M, X, Y, boot90_bounds)
    true_boot90_po_sales = np.sum(sales_function(boot90_po_prices, alpha, beta))
    boot90_sales_list_5.append(boot90_po_sales / so_sales)
    true_boot90_sales_list_5.append(true_boot90_po_sales / so_sales)
    boot90_range = sum(boot90_bounds[i][1] - boot90_bounds[i][0] for i in range(M))
    boot90_range_list_5.append(boot90_range)


    # EBZ
    ebz_val, ebz_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.6,
        1.0,
        1.0,
    )
    ebz_po_sales, ebz_po_prices = predict_optimize(M, X, Y, ebz_bounds)
    true_ebz_po_sales = np.sum(sales_function(ebz_po_prices, alpha, beta))
    ebz_po_sales_list_5.append(ebz_po_sales / so_sales)
    true_ebz_po_sales_list_5.append(true_ebz_po_sales / so_sales)
    ebz_range = sum(ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M))
    ebz_range_list_5.append(ebz_range)

    # EBPA(0.3)
    ebpa3_val, ebpa3_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.30,
        1.0,
        1.0,
    )
    ebpa3_po_sales, ebpa3_po_prices = predict_optimize(M, X, Y, ebpa3_bounds)
    true_ebpa3_po_sales = np.sum(sales_function(ebpa3_po_prices, alpha, beta))
    ebpa3_po_sales_list_5.append(ebpa3_po_sales / so_sales)
    true_ebpa3_po_sales_list_5.append(true_ebpa3_po_sales / so_sales)
    ebpa3_range = sum(ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M))
    ebpa3_range_list_5.append(ebpa3_range)

    # EBPA(0.4)
    ebpa4_val, ebpa4_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.40,
        1.0,
        1.0,
    )
    ebpa4_po_sales, ebpa4_po_prices = predict_optimize(M, X, Y, ebpa4_bounds)
    true_ebpa4_po_sales = np.sum(sales_function(ebpa4_po_prices, alpha, beta))
    ebpa4_po_sales_list_5.append(ebpa4_po_sales / so_sales)
    true_ebpa4_po_sales_list_5.append(true_ebpa4_po_sales / so_sales)
    ebpa4_range = sum(ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M))
    ebpa4_range_list_5.append(ebpa4_range)

    # EBPA(0.5)
    ebpa5_val, ebpa5_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.50,
        1.0,
        1.0,
    )
    ebpa5_po_sales, ebpa5_po_prices = predict_optimize(M, X, Y, ebpa5_bounds)
    true_ebpa5_po_sales = np.sum(sales_function(ebpa5_po_prices, alpha, beta))
    ebpa5_po_sales_list_5.append(ebpa5_po_sales / so_sales)
    true_ebpa5_po_sales_list_5.append(true_ebpa5_po_sales / so_sales)
    ebpa5_range = sum(ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M))
    ebpa5_range_list_5.append(ebpa5_range)

    ebz_range_deff_list_ = [ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M)]
    ebpa3_range_deff_list_ = [ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M)]
    ebpa4_range_deff_list_ = [ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M)]
    ebpa5_range_deff_list_ = [ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M)]

    ebz_range_deff_list_5.append(np.sum(ebz_range_deff_list_))
    ebpa3_range_deff_list_5.append(np.sum(ebpa3_range_deff_list_))
    ebpa4_range_deff_list_5.append(np.sum(ebpa4_range_deff_list_))
    ebpa5_range_deff_list_5.append(np.sum(ebpa5_range_deff_list_))

    test_lr = MultiOutputRegressor(LinearRegression())
    test_lr.fit(X, Y)
    test_coefs = [estimate.coef_ for estimate in test_lr.estimators_]
    test_intercepts = [estimate.intercept_ for estimate in test_lr.estimators_]

    all_pred = test_lr.predict(X)
    for k, estimator in enumerate(test_lr.estimators_):
        # k番目の列（= k番目の出力）の真の値と予測値を取り出し
        y_true_k = Y[:, k]
        y_pred_k = all_pred[:, k]
        # print(f"y_true_k: {y_true_k}")
        # print(f"y_pred_k: {y_pred_k}")
        # print(y_true_k == y_pred_k)
        # print(estimator)
        # RMSE の計算
        # R^2 の計算
        r2_k = r2_score(y_true_k, y_pred_k)

        all_r2_list.append(r2_k)

    # ★ 追加：各商品の範囲の差を記録
    for j in range(M):
        each_range_deff_6_list.append(ebz_bounds[j][1] - ebz_bounds[j][0])
        each_range_deff_3_list.append(ebpa3_bounds[j][1] - ebpa3_bounds[j][0])
        each_range_deff_4_list.append(ebpa4_bounds[j][1] - ebpa4_bounds[j][0])
        each_range_deff_5_list.append(ebpa5_bounds[j][1] - ebpa5_bounds[j][0])

# -------------------------------
# 最後に全体データ学習モデルの R^2 を確認

In [ ]:
# df_5_6にso_sales_list_5からebpa_range_diff_5までのリストを追加

df_5_8 = pd.DataFrame(
    {
        "so_sales_list_5": so_sales_list_5,
        "po_sales_list_5": po_sales_list_5,
        "true_po_sales_list_5": true_po_sales_list_5,
        "ebz_po_sales_list_5": ebz_po_sales_list_5,
        "true_ebz_po_sales_list_5": true_ebz_po_sales_list_5,
        "quan95_sales_list_5": quan95_sales_list_5,
        "true_quan95_sales_list_5": true_quan95_sales_list_5,
        "quan90_sales_list_5": quan90_sales_list_5,
        "true_quan90_sales_list_5": true_quan90_sales_list_5,
        "quan85_sales_list_5": quan85_sales_list_5,
        "true_quan85_sales_list_5": true_quan85_sales_list_5,
        "quan80_sales_list_5": quan80_sales_list_5,
        "true_quan80_sales_list_5": true_quan80_sales_list_5,
        "boot99_sales_list_5": boot99_sales_list_5,
        "true_boot99_sales_list_5": true_boot99_sales_list_5,
        "boot95_sales_list_5": boot95_sales_list_5,
        "true_boot95_sales_list_5": true_boot95_sales_list_5,
        "boot90_sales_list_5": boot90_sales_list_5,
        "true_boot90_sales_list_5": true_boot90_sales_list_5,
        "ebpa3_po_sales_list_5": ebpa3_po_sales_list_5,
        "true_ebpa3_po_sales_list_5": true_ebpa3_po_sales_list_5,
        "ebpa4_po_sales_list_5": ebpa4_po_sales_list_5,
        "true_ebpa4_po_sales_list_5": true_ebpa4_po_sales_list_5,
        "ebpa5_po_sales_list_5": ebpa5_po_sales_list_5,
        "true_ebpa5_po_sales_list_5": true_ebpa5_po_sales_list_5,
        "epa6_po_sales_list_5": ebz_po_sales_list_5,
        "true_ebpa6_po_sales_list_5": true_ebz_po_sales_list_5,
        "ebpa3_range_deff_list_5": ebpa3_range_deff_list_5,
        "ebpa4_range_deff_list_5": ebpa4_range_deff_list_5,
        "ebpa5_range_deff_list_5": ebpa5_range_deff_list_5,
        "ebpa6_range_deff_list_5": ebz_range_deff_list_5,
    }
)

# csvファイルに出力
df_5_8.to_csv("df_5_8_2.csv", index=False)

In [ ]:
df_5_8_range = pd.DataFrame(
    {
    "quan95_range_list_5": quan95_range_list_5,
    "quan90_range_list_5": quan90_range_list_5,
    "quan85_range_list_5": quan85_range_list_5,
    "quan80_range_list_5": quan80_range_list_5,
    "boot99_range_list_5": boot99_range_list_5,
    "boot95_range_list_5": boot95_range_list_5,
    "boot90_range_list_5": boot90_range_list_5,
    "ebz_range_list_5": ebz_range_list_5,
    "ebpa3_range_list_5": ebpa3_range_list_5,
    "ebpa4_range_list_5": ebpa4_range_list_5,
    "ebpa5_range_list_5": ebpa5_range_list_5,
    }
)
# csvファイルに出力
df_5_8_range.to_csv("df_5_8_range_2.csv", index=False)

In [ ]:
df_5_8_each = pd.DataFrame(
    {
        "R2": all_r2_list,
        "each_range_deff_3_list": each_range_deff_3_list,
        "each_range_deff_4_list": each_range_deff_4_list,
        "each_range_deff_5_list": each_range_deff_5_list,
        "each_range_deff_6_list": each_range_deff_6_list,
    }
)

# csvファイルに出力
df_5_8_each.to_csv("df_5_8_each_2.csv", index=False)

In [ ]:
# ----- 以下はお使いの既存の関数群を想定 -----
# create_date, create_bounds, sales_optimize, predict_optimize,
# sales_function, bound_quan, estimate_bounds_penalty_nelder_all,
# cross_validation などは、既に定義済みとして進めます。

# 実験設定
M = 5
K = 5
N = 500
B=100
r_mean = 0.8
r_std = 0.1
r_min = 0.5
r_max = 1.1
delta = 1.0
z_range = 0.4

lb, ub, bounds, range_bounds = create_bounds(M, r_min, r_max)

# 実験を何回行うか（例：2回）
num_experiments = 100

so_sales_list_5 = []
po_sales_list_5 = []
true_po_sales_list_5 = []

ebz_po_sales_list_5 = []
true_ebz_po_sales_list_5 = []
ebz_range_list_5 = []

quan95_sales_list_5 = []
true_quan95_sales_list_5 = []
quan95_range_list_5 = []

quan90_sales_list_5 = []
true_quan90_sales_list_5 = []
quan90_range_list_5 = []

quan85_sales_list_5 = []
true_quan85_sales_list_5 = []
quan85_range_list_5 = []

quan80_sales_list_5 = []
true_quan80_sales_list_5 = []
quan80_range_list_5 = []

boot99_sales_list_5 = []
true_boot99_sales_list_5 = []
boot99_range_list_5 = []

boot95_sales_list_5 = []
true_boot95_sales_list_5 = []
boot95_range_list_5 = []

boot90_sales_list_5 = []
true_boot90_sales_list_5 = []
boot90_range_list_5 = []


ebpa3_po_sales_list_5 = []
true_ebpa3_po_sales_list_5 = []
ebpa3_range_list_5 = []

ebpa4_po_sales_list_5 = []
true_ebpa4_po_sales_list_5 = []
ebpa4_range_list_5 = []

ebpa5_po_sales_list_5 = []
true_ebpa5_po_sales_list_5 = [] 
ebpa5_range_list_5 = []

ebz_range_deff_list_5 = []
ebpa3_range_deff_list_5 = []
ebpa4_range_deff_list_5 = []
ebpa5_range_deff_list_5 = []

# ★ 追加： 全体学習モデルの R^2 を記録するリスト
all_r2_list = []

each_range_deff_3_list = []
each_range_deff_4_list = []
each_range_deff_5_list = []
each_range_deff_6_list = []

for i in range(num_experiments):
    np.random.seed(i + 10)
    alpha, beta, X, Y = create_date(M, N, r_mean, r_std, delta)

    tilda_coefs_list = []
    tilda_intercepts_list = []
    hat_coefs_list = []
    hat_intercepts_list = []

    # ----------- 1) KFold で tilda / hat を推定 (境界推定用) -----------
    kf = KFold(n_splits=K, shuffle=True, random_state=0)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        # lr_tilda: trainデータで学習
        lr_tilda = MultiOutputRegressor(LinearRegression())
        lr_tilda.fit(X_train, y_train)

        # tildaの係数・切片を保存
        coefs = [est.coef_ for est in lr_tilda.estimators_]
        intercepts = [est.intercept_ for est in lr_tilda.estimators_]
        tilda_coefs_list.append(coefs)
        tilda_intercepts_list.append(intercepts)

        # lr_hat: testデータで学習
        lr_hat = MultiOutputRegressor(LinearRegression())
        lr_hat.fit(X_test, y_test)

        hat_coefs = [est.coef_ for est in lr_hat.estimators_]
        hat_intercepts = [est.intercept_ for est in lr_hat.estimators_]
        hat_coefs_list.append(hat_coefs)
        hat_intercepts_list.append(hat_intercepts)

    # ----------- 3) 以下、既存の最適化ロジック -----------
    # SO (真のパラメータを使った最適化)
    so_sales, so_prices = sales_optimize(M, alpha, beta, bounds)

    # PO (実データ X, Y で学習したモデルを使い最適化)
    po_sales, po_prices = predict_optimize(M, X, Y, bounds)
    true_po_sales = np.sum(sales_function(po_prices, alpha, beta))

    so_sales_list_5.append(so_sales / so_sales)
    po_sales_list_5.append(po_sales / so_sales)
    true_po_sales_list_5.append(true_po_sales / so_sales)

    # QUAN(95%)
    quan95_bounds = bound_quan(X, 0.95)
    quan95_po_sales, quan95_po_prices = predict_optimize(M, X, Y, quan95_bounds)
    true_quan95_po_sales = np.sum(sales_function(quan95_po_prices, alpha, beta))
    quan95_sales_list_5.append(quan95_po_sales / so_sales)
    true_quan95_sales_list_5.append(true_quan95_po_sales / so_sales)
    quan95_range = sum(quan95_bounds[i][1] - quan95_bounds[i][0] for i in range(M))
    quan95_range_list_5.append(quan95_range)

    # QUAN(90%)
    quan90_bounds = bound_quan(X, 0.9)
    quan90_po_sales, quan90_po_prices = predict_optimize(M, X, Y, quan90_bounds)
    true_quan90_po_sales = np.sum(sales_function(quan90_po_prices, alpha, beta))
    quan90_sales_list_5.append(quan90_po_sales / so_sales)
    true_quan90_sales_list_5.append(true_quan90_po_sales / so_sales)
    quan90_range = sum(quan90_bounds[i][1] - quan90_bounds[i][0] for i in range(M))
    quan90_range_list_5.append(quan90_range)

    # QUAN(85%)
    quan85_bounds = bound_quan(X, 0.85)
    quan85_po_sales, quan85_po_prices = predict_optimize(M, X, Y, quan85_bounds)
    true_quan85_po_sales = np.sum(sales_function(quan85_po_prices, alpha, beta))
    quan85_sales_list_5.append(quan85_po_sales / so_sales)
    true_quan85_sales_list_5.append(true_quan85_po_sales / so_sales)
    quan85_range = sum(quan85_bounds[i][1] - quan85_bounds[i][0] for i in range(M))
    quan85_range_list_5.append(quan85_range)

    # QUAN(80%)
    quan80_bounds = bound_quan(X, 0.8)
    quan80_po_sales, quan80_po_prices = predict_optimize(M, X, Y, quan80_bounds)
    true_quan80_po_sales = np.sum(sales_function(quan80_po_prices, alpha, beta))
    quan80_sales_list_5.append(quan80_po_sales / so_sales)
    true_quan80_sales_list_5.append(true_quan80_po_sales / so_sales)
    quan80_range = sum(quan80_bounds[i][1] - quan80_bounds[i][0] for i in range(M))
    quan80_range_list_5.append(quan80_range)

    # BOOt(99%)
    boot99_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=2.576
    )
    boot99_po_sales, boot99_po_prices = predict_optimize(M, X, Y, boot99_bounds)
    true_boot99_po_sales = np.sum(sales_function(boot99_po_prices, alpha, beta))
    boot99_sales_list_5.append(boot99_po_sales / so_sales)
    true_boot99_sales_list_5.append(true_boot99_po_sales / so_sales)
    boot99_range = sum(boot99_bounds[i][1] - boot99_bounds[i][0] for i in range(M))
    boot99_range_list_5.append(boot99_range)

    # boot(95%)
    boot95_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=1.96
    )
    boot95_po_sales, boot95_po_prices = predict_optimize(M, X, Y, boot95_bounds)
    true_boot95_po_sales = np.sum(sales_function(boot95_po_prices, alpha, beta))
    boot95_sales_list_5.append(boot95_po_sales / so_sales)
    true_quan95_sales_list_5.append(true_boot95_po_sales / so_sales)
    boot95_range = sum(boot95_bounds[i][1] - boot95_bounds[i][0] for i in range(M))
    boot95_range_list_5.append(boot95_range)

    # boot(90%)
    boot90_bounds = bootstrap_bounds(
        M, X, Y, r_min, r_max, n_iterations=B, k=1.645
    )
    boot90_po_sales, boot90_po_prices = predict_optimize(M, X, Y, boot90_bounds)
    true_boot90_po_sales = np.sum(sales_function(boot90_po_prices, alpha, beta))
    boot90_sales_list_5.append(boot90_po_sales / so_sales)
    true_boot90_sales_list_5.append(true_boot90_po_sales / so_sales)
    boot90_range = sum(boot90_bounds[i][1] - boot90_bounds[i][0] for i in range(M))
    boot90_range_list_5.append(boot90_range)


    # EBZ
    ebz_val, ebz_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.6,
        1.0,
        1.0,
    )
    ebz_po_sales, ebz_po_prices = predict_optimize(M, X, Y, ebz_bounds)
    true_ebz_po_sales = np.sum(sales_function(ebz_po_prices, alpha, beta))
    ebz_po_sales_list_5.append(ebz_po_sales / so_sales)
    true_ebz_po_sales_list_5.append(true_ebz_po_sales / so_sales)
    ebz_range = sum(ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M))
    ebz_range_list_5.append(ebz_range)

    # EBPA(0.3)
    ebpa3_val, ebpa3_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.30,
        1.0,
        1.0,
    )
    ebpa3_po_sales, ebpa3_po_prices = predict_optimize(M, X, Y, ebpa3_bounds)
    true_ebpa3_po_sales = np.sum(sales_function(ebpa3_po_prices, alpha, beta))
    ebpa3_po_sales_list_5.append(ebpa3_po_sales / so_sales)
    true_ebpa3_po_sales_list_5.append(true_ebpa3_po_sales / so_sales)
    ebpa3_range = sum(ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M))
    ebpa3_range_list_5.append(ebpa3_range)

    # EBPA(0.4)
    ebpa4_val, ebpa4_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.40,
        1.0,
        1.0,
    )
    ebpa4_po_sales, ebpa4_po_prices = predict_optimize(M, X, Y, ebpa4_bounds)
    true_ebpa4_po_sales = np.sum(sales_function(ebpa4_po_prices, alpha, beta))
    ebpa4_po_sales_list_5.append(ebpa4_po_sales / so_sales)
    true_ebpa4_po_sales_list_5.append(true_ebpa4_po_sales / so_sales)
    ebpa4_range = sum(ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M))
    ebpa4_range_list_5.append(ebpa4_range)

    # EBPA(0.5)
    ebpa5_val, ebpa5_bounds = estimate_bounds_penalty_nelder_all(
        range_bounds,
        tilda_coefs_list,
        tilda_intercepts_list,
        hat_coefs_list,
        hat_intercepts_list,
        M,
        K,
        r_min,
        r_max,
        0.50,
        1.0,
        1.0,
    )
    ebpa5_po_sales, ebpa5_po_prices = predict_optimize(M, X, Y, ebpa5_bounds)
    true_ebpa5_po_sales = np.sum(sales_function(ebpa5_po_prices, alpha, beta))
    ebpa5_po_sales_list_5.append(ebpa5_po_sales / so_sales)
    true_ebpa5_po_sales_list_5.append(true_ebpa5_po_sales / so_sales)
    ebpa5_range = sum(ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M))
    ebpa5_range_list_5.append(ebpa5_range)

    ebz_range_deff_list_ = [ebz_bounds[i][1] - ebz_bounds[i][0] for i in range(M)]
    ebpa3_range_deff_list_ = [ebpa3_bounds[i][1] - ebpa3_bounds[i][0] for i in range(M)]
    ebpa4_range_deff_list_ = [ebpa4_bounds[i][1] - ebpa4_bounds[i][0] for i in range(M)]
    ebpa5_range_deff_list_ = [ebpa5_bounds[i][1] - ebpa5_bounds[i][0] for i in range(M)]

    ebz_range_deff_list_5.append(np.sum(ebz_range_deff_list_))
    ebpa3_range_deff_list_5.append(np.sum(ebpa3_range_deff_list_))
    ebpa4_range_deff_list_5.append(np.sum(ebpa4_range_deff_list_))
    ebpa5_range_deff_list_5.append(np.sum(ebpa5_range_deff_list_))

    test_lr = MultiOutputRegressor(LinearRegression())
    test_lr.fit(X, Y)
    test_coefs = [estimate.coef_ for estimate in test_lr.estimators_]
    test_intercepts = [estimate.intercept_ for estimate in test_lr.estimators_]

    all_pred = test_lr.predict(X)
    for k, estimator in enumerate(test_lr.estimators_):
        # k番目の列（= k番目の出力）の真の値と予測値を取り出し
        y_true_k = Y[:, k]
        y_pred_k = all_pred[:, k]
        # print(f"y_true_k: {y_true_k}")
        # print(f"y_pred_k: {y_pred_k}")
        # print(y_true_k == y_pred_k)
        # print(estimator)
        # RMSE の計算
        # R^2 の計算
        r2_k = r2_score(y_true_k, y_pred_k)

        all_r2_list.append(r2_k)

    # ★ 追加：各商品の範囲の差を記録
    for j in range(M):
        each_range_deff_6_list.append(ebz_bounds[j][1] - ebz_bounds[j][0])
        each_range_deff_3_list.append(ebpa3_bounds[j][1] - ebpa3_bounds[j][0])
        each_range_deff_4_list.append(ebpa4_bounds[j][1] - ebpa4_bounds[j][0])
        each_range_deff_5_list.append(ebpa5_bounds[j][1] - ebpa5_bounds[j][0])

# -------------------------------
# 最後に全体データ学習モデルの R^2 を確認

In [ ]:
# df_5_6にso_sales_list_5からebpa_range_diff_5までのリストを追加

df_5_10 = pd.DataFrame(
    {
        "so_sales_list_5": so_sales_list_5,
        "po_sales_list_5": po_sales_list_5,
        "true_po_sales_list_5": true_po_sales_list_5,
        "ebz_po_sales_list_5": ebz_po_sales_list_5,
        "true_ebz_po_sales_list_5": true_ebz_po_sales_list_5,
        "quan95_sales_list_5": quan95_sales_list_5,
        "true_quan95_sales_list_5": true_quan95_sales_list_5,
        "quan90_sales_list_5": quan90_sales_list_5,
        "true_quan90_sales_list_5": true_quan90_sales_list_5,
        "quan85_sales_list_5": quan85_sales_list_5,
        "true_quan85_sales_list_5": true_quan85_sales_list_5,
        "quan80_sales_list_5": quan80_sales_list_5,
        "true_quan80_sales_list_5": true_quan80_sales_list_5,
        "boot99_sales_list_5": boot99_sales_list_5,
        "true_boot99_sales_list_5": true_boot99_sales_list_5,
        "boot95_sales_list_5": boot95_sales_list_5,
        "true_boot95_sales_list_5": true_boot95_sales_list_5,
        "boot90_sales_list_5": boot90_sales_list_5,
        "true_boot90_sales_list_5": true_boot90_sales_list_5,
        "ebpa3_po_sales_list_5": ebpa3_po_sales_list_5,
        "true_ebpa3_po_sales_list_5": true_ebpa3_po_sales_list_5,
        "ebpa4_po_sales_list_5": ebpa4_po_sales_list_5,
        "true_ebpa4_po_sales_list_5": true_ebpa4_po_sales_list_5,
        "ebpa5_po_sales_list_5": ebpa5_po_sales_list_5,
        "true_ebpa5_po_sales_list_5": true_ebpa5_po_sales_list_5,
        "epa6_po_sales_list_5": ebz_po_sales_list_5,
        "true_ebpa6_po_sales_list_5": true_ebz_po_sales_list_5,
        "ebpa3_range_deff_list_5": ebpa3_range_deff_list_5,
        "ebpa4_range_deff_list_5": ebpa4_range_deff_list_5,
        "ebpa5_range_deff_list_5": ebpa5_range_deff_list_5,
        "ebpa6_range_deff_list_5": ebz_range_deff_list_5,
    }
)

# csvファイルに出力
df_5_10.to_csv("df_5_10_2.csv", index=False)

In [ ]:
df_5_10_range = pd.DataFrame(
    {
    "quan95_range_list_5": quan95_range_list_5,
    "quan90_range_list_5": quan90_range_list_5,
    "quan85_range_list_5": quan85_range_list_5,
    "quan80_range_list_5": quan80_range_list_5,
    "boot99_range_list_5": boot99_range_list_5,
    "boot95_range_list_5": boot95_range_list_5,
    "boot90_range_list_5": boot90_range_list_5,
    "ebz_range_list_5": ebz_range_list_5,
    "ebpa3_range_list_5": ebpa3_range_list_5,
    "ebpa4_range_list_5": ebpa4_range_list_5,
    "ebpa5_range_list_5": ebpa5_range_list_5,
    }
)
# csvファイルに出力
df_5_10_range.to_csv("df_5_10_range_2.csv", index=False)

In [ ]:
df_5_10_each = pd.DataFrame(
    {
        "R2": all_r2_list,
        "each_range_deff_3_list": each_range_deff_3_list,
        "each_range_deff_4_list": each_range_deff_4_list,
        "each_range_deff_5_list": each_range_deff_5_list,
        "each_range_deff_6_list": each_range_deff_6_list,
    }
)

# csvファイルに出力
df_5_10_each.to_csv("df_5_10_each_2.csv", index=False)